In [73]:
import numpy as np
import pandas as pd
from math import log
from sklearn import datasets
from collections import Counter

In [74]:
iris = datasets.load_iris()

In [75]:
df = pd.DataFrame(iris.data)
df.columns = ["sl", "sw", 'pl', 'pw']

In [76]:
#Function to find label for a value
#if MIN_Value <=val < (m + Mean_Value) / 2 then it is assigned label a
#if (m + Mean_Value) <=val < Mean_Value then it is assigned label b
#if (Mean_Value) <=val < (Mean_Value + MAX_Value)/2 then it is assigned label c
#if (Mean_Value + MAX_Value)/2 <=val <= MAX_Value  then it is assigned label d

def label(val, *boundaries):
    if (val < boundaries[0]):
        return 'a'
    elif (val < boundaries[1]):
        return 'b'
    elif (val < boundaries[2]):
        return 'c'
    else:
        return 'd'

#Function to convert a continuous data into labelled data
#There are 4 lables  - a, b, c, d
def toLabel(df, old_feature_name):
    second = df[old_feature_name].mean()
    minimum = df[old_feature_name].min()
    first = (minimum + second)/2
    maximum = df[old_feature_name].max()
    third = (maximum + second)/2
    return df[old_feature_name].apply(label, args= (first, second, third))

In [77]:
#Convert all columns to labelled data
df['sl_labeled'] = toLabel(df, 'sl')
df['sw_labeled'] = toLabel(df, 'sw')
df['pl_labeled'] = toLabel(df, 'pl')
df['pw_labeled'] = toLabel(df, 'pw')
df

,sl,sw,pl,pw,sl_labeled,sw_labeled,pl_labeled,pw_labeled
0,5.1,3.5,1.4,0.2,b,c,a,a
1,4.9,3.0,1.4,0.2,a,b,a,a
2,4.7,3.2,1.3,0.2,a,c,a,a
3,4.6,3.1,1.5,0.2,a,c,a,a
4,5.0,3.6,1.4,0.2,a,c,a,a
...,...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,c,b,c,d
146,6.3,2.5,5.0,1.9,c,a,c,d
147,6.5,3.0,5.2,2.0,c,b,c,d
148,6.2,3.4,5.4,2.3,c,c,d,d


In [78]:
df.drop(['sl', 'sw', 'pl', 'pw'], axis = 1, inplace = True)

In [79]:
df

,sl_labeled,sw_labeled,pl_labeled,pw_labeled
0,b,c,a,a
1,a,b,a,a
2,a,c,a,a
3,a,c,a,a
4,a,c,a,a
...,...,...,...,...
145,c,b,c,d
146,c,a,c,d
147,c,b,c,d
148,c,c,d,d


In [80]:
set(df['sl_labeled'])

{'a', 'b', 'c', 'd'}

In [81]:
def Entropy(y):
    '''
    StraightForward
    '''
    ans = 0
    length = len(y)
    counter = Counter(y)
    for i in counter:
        p = counter[i]/length
        ans += p*log(p)
    if ans == 0:
        return 0
    return -ans

In [82]:
def gain_ratio(info_gain,indices):
    '''
    Find split info and divide info gain with split info to find GAIN RATIO
    '''
    split_info = 0
    D = len(indices[0])
    for ind in indices:
        d = ind.sum()
        di = d/D
        split_info += di*log(di)
    
    return info_gain / (-split_info)


In [83]:
class TreeNode:
    
    def __init__(self):
        
        self.children = []
        self.entropy = 0
        self.count_of_classes = []


In [84]:
def build_tree_for_real(root,df, y, unused_features,lvl):
    
    counter = Counter(y)
    
    
    
    if len(unused_features) == 0 or len(set(y)) == 1:

        for i in counter:
             
            root.count_of_classes.append([i,counter[i]])
        

        root.entropy = Entropy(y)  ## storing entropy of this root
        
        
        return root
    
    
    info_this = Entropy(y) 
    
    root.entropy = info_this

    for i in counter:
        root.count_of_classes.append([i,counter[i]])  ## storing children of this root
    
    
        
    best_feature = ""
    min_info = 123234
    final_indices  = ''
    for f in unused_features:
        possible_values = set(df[f])
        
        indices = []
        for i in possible_values:
            ind = df[f] == i
            indices.append(ind)
        
        info_split = 0
        for ind in indices:
            y_label_in_feature = y[ind]
            info_split += Entropy(y_label_in_feature)*(ind.sum())
            
        info_split = info_split / len(y)
        
        if info_split < min_info:
            min_info = info_split
            best_feature = f
            final_indices = indices
    
    info_gain = info_this - min_info


    
    unused_features.remove(best_feature)
    

    for ind in final_indices:
        
        root_child  = TreeNode()
        root.children.append(build_tree_for_real(root_child,df[ind].copy(),y[ind].copy(),unused_features.copy(),lvl+1)) 
        
        
    
    return root
   

In [88]:

def print_level_traversal(root):
    
    nodes = []
    nodes.append(root)
    while len(nodes) != 0:
        root= nodes.pop()
        for i in root.count_of_classes:
            print("Count of Class {0} : {1}".format(i[0],i[1]))
        
        print('Entropy : {}'.format(root.entropy))
        for i in root.children:
            nodes.append(i)
        print()
        

In [89]:
root = TreeNode()
root= build_tree_for_real(root,df.copy(), y.copy(), unused_features.copy(),0) ## root of tree


In [90]:
print_level_traversal(root)

Count of Class 0 : 50
Count of Class 1 : 50
Count of Class 2 : 50
Entropy : 1.0986122886681096

Count of Class 1 : 10
Entropy : 0

Count of Class 2 : 34
Entropy : 0

Count of Class 0 : 50
Entropy : 0

Count of Class 1 : 40
Count of Class 2 : 16
Entropy : 0.5982695885852573

Count of Class 1 : 1
Entropy : 0

Count of Class 2 : 8
Entropy : 0

Count of Class 1 : 39
Count of Class 2 : 8
Entropy : 0.45622342016761397

Count of Class 1 : 14
Entropy : 0

Count of Class 1 : 2
Entropy : 0

Count of Class 1 : 23
Count of Class 2 : 7
Entropy : 0.5432727813369008

Count of Class 1 : 14
Count of Class 2 : 6
Entropy : 0.6108643020548935

Count of Class 1 : 3
Count of Class 2 : 1
Entropy : 0.5623351446188083

Count of Class 1 : 6
Entropy : 0

Count of Class 2 : 1
Entropy : 0

